In [ ]:
import cma,torch,os
import numpy as np
os.environ['KMP_DUPLICATE_LIB_OK']='True'


best_value = 1000
best_position = None
res_img = None

# 定义目标函数，这里只是一个示例函数
def objective_function(x):
    global best_value, best_position, res_img
    x = torch.tensor(x)
    res = registration.similarity(x)
    fitness = -res[0]
    if fitness < best_value:
        best_value = fitness
        best_position = x
        res_img = res[1]
    return fitness

init_translate = registration.config.init_translate
translate_delta = registration.config.translate_delta
center_point = [init_translate[0]+translate_delta[0]/2, init_translate[1]+translate_delta[1]/2]

# 定义优化参数
initial_guess = np.array([center_point[0], center_point[1], 180])  # 初始猜测解，根据问题维度调整

bounds = np.array((pso.minV, pso.maxV))

sigma0 = 100  # 初始标准差，表示搜索范围的大小
popsize = 256 #pso.particle_num  # 种群大小

# 初始化 CMA-ES 优化器
es = cma.CMAEvolutionStrategy(initial_guess, sigma0, {'popsize': popsize, 
                                                      'bounds':[bounds[0], bounds[1]],
                                                      'maxiter':50,
                                                      'verb_disp': 1,    # 每代打印一次信息
                                                      })

# 开始优化过程
while not es.stop():
    # 获取一组新的样本点
    solutions = es.ask()
    
    # 计算每个样本点的目标函数值
    function_values = [objective_function(solution) for solution in solutions]
    
    print(f"iterations solution found: {best_position}")
    print(f"iterations fitness achieved: {-best_value}")

    # 将目标函数值反馈给 CMA-ES
    es.tell(solutions, function_values)

    # 输出当前的状态信息（可选）
    es.logger.add()  # write data to logger

# 获取最终的最佳解
best_solution = es.result.xbest
best_fitness = es.result.fbest

print(f"Best solution found: {best_solution}")
print(f"Best fitness achieved: {best_fitness}")